In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

### 1단계: 문서 분할 설정

In [7]:
# 문서 로딩
print("==> 문서 로딩 → PDF 사례집 읽기...")
loader = PyPDFLoader('../data/dispute-cases.pdf')
documents = loader.load()

#문서 분할
print("==> 문서 분할 → 법률 사례 단위로 청크 분할")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", 
        "\n【쟁점사항】",
        "\n【처리경위】", 
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

chunks = text_splitter.split_documents(documents)
print(f" 분할된 청크 개수: {len(chunks)}")


==> 문서 로딩 → PDF 사례집 읽기...
==> 문서 분할 → 법률 사례 단위로 청크 분할
 분할된 청크 개수: 104


### 2단계: 임베딩 모델 설정

In [8]:
print("==>  벡터화 → 임베딩으로 변환")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1536
)

==>  벡터화 → 임베딩으로 변환


### 3단계: 검색기 설정

In [9]:
# 벡터스토어 생성
print("==>  저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

# 검색기 생성
print("===> 검색 → 질문과 유사한 분쟁 사례 검색")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)
print(" Retriever 설정 완료")

==>  저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (104개 벡터)
===> 검색 → 질문과 유사한 분쟁 사례 검색
 Retriever 설정 완료


### 4단계: LLM 설정

In [10]:
# LLM + 프롬프트 설정
print("===> 생성 → LLM으로 답변 생성")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

===> 생성 → LLM으로 답변 생성


### 5단계: 법률 자문 프롬프트 작성

In [11]:
# 한국어 최적화 프롬프트
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

print(" 프롬프트 설정 완료")

 프롬프트 설정 완료


### 6단계: QA 체인 생성

In [12]:
# QA 체인 생성
print("\n ===>  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("  RAG 파이프라인 구축 완료!")


 ===>  QA 체인 생성...
  RAG 파이프라인 구축 완료!


### 7단계: 테스트 질문 작성

In [13]:
# 테스트 질문
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

print("\n" + "=" * 60)
print(" 콘텐츠 분쟁 해결 RAG 시스템 테스트")
print("=" * 60)


 콘텐츠 분쟁 해결 RAG 시스템 테스트


### 8단계: 분쟁 유형 분류 함수

In [14]:
# 분쟁 유형 분류 함수
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"


### 테스트

In [15]:
# 질문 및 답변 실행
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")

    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]

    print(f"\n 답변:")
    print("-" * 50)
    print(answer)

    print(f"\n 참조 사례:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")

    print("\n" + "-" * 40)


【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 근거로 다음과 같이 답변드립니다.

1. **사례 분석**:
   - **사례 1 (2006년)**: 시스템 오류로 아이템이 소멸된 경우, 계정 명의자가 아닌 신청인의 복구 요청은 거부되었습니다. 게임사의 약관에 따라 계정 명의자가 직접 복구 신청을 해야 한다는 점이 강조되었습니다.
   - **사례 2 (2009년)**: 시스템 오류로 인한 게임머니 소실 주장이 있었으나, 유사한 불만이 접수되지 않았고, 사용 내역상 문제점이 발견되지 않아 복구가 거부되었습니다.
   - **사례 5 (2008년)**: 보안 서비스 미작동으로 인한 아이템 도난의 경우, 게임사가 도난을 인정하고 아이템을 복구해 주었습니다.

2. **관련 법령 및 조항**:
   - 제시된 사례에서는 민법 제250조 및 제251조가 언급되었으나, 이는 도난 및 유실물에 관한 조항으로, 시스템 오류와 직접적인 관련은 없습니다. 시스템 오류에 대한 법적 책임은 주로 계약법 및 소비자 보호법에 따라 판단될 수 있습니다.

3. **실무적 해결방안**:
   - **계정 명의 확인**: 계정 명의자가 본인인지 확인하고, 명의자가 아닌 경우 명의자에게 복구 요청을 의뢰합니다.
   - **증거 수집**: 시스템 오류로 인한 아이템 소실을 증명할 수 있는 증거(스크린샷, 로그 기록 등)를 수집합니다.
   - **게임사와의 협상**: 수집한 증거를 바탕으로 게임사에 다시 복구 요청을 하고, 게임사의 약관 및 정책을 검토하여 협상합니다.
   - **소비자 보호 기관 상담**: 한국소비자원 등 관련 기관